In [2]:
import pandas as pd
from pandas import ExcelWriter

import numpy as np
import os
import io

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
import warnings
 
warnings.filterwarnings('ignore')

#import dash
#from dash import html, Dash, dash_table, dcc, callback, Output, Input
#import dash_bootstrap_components as dibc


from base64 import b64encode

import re

In [7]:
df_arranques = pd.read_excel(r'data\csv\CIERRE_INTERRUPTOR 1.xlsx')
column_names = df_arranques.columns
print(column_names)  # Output: Index(['Date', 'Price', 'Volume'], dtype='object')

Index(['ARRANQUE', 'CIERRE INTERRUPTOR', 'INICIO (-0,5 hs)', 'FIN (+2 hs)'], dtype='object')


In [10]:
# Crear un diccionario para almacenar los DataFrames
dataframes = {}
folder_path = fr"data\csv"

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df_name = filename[:-4]  # Nombre del DataFrame basado en el nombre del archivo
        dataframes[df_name] = pd.read_csv(file_path, names=['*TG05*', 'Estampa de Tiempo', 'Variable'])

# Acceder a los DataFrames generados
for name, df in dataframes.items():
    print(f'DataFrame: {name}')

DataFrame: TG5-LL-BNA5.BNA_XE8132A_PT_4
DataFrame: TG5-LL-BNA5.BNA_XE8134A_2
DataFrame: TG5-LL-BNA5.BNA_XE8134B_2
DataFrame: TG5-LL-BNA5.BNA_XE8134C_2
DataFrame: TG5-LL-BNA5.BNA_XE8135A_2
DataFrame: TG5-LL-BNA5.BNA_XE8135B_2
DataFrame: TG5-LL-BNA5.BNA_XE8135C_2
DataFrame: TG5-LL-BNB5.BNB_XE8131B_PT_4
DataFrame: TG5-LL-BNB5.BNB_XE8133B_PT_4
DataFrame: TG5-LL-BNC5.BNC_XE8138X1_1
DataFrame: TG5-LL-BNC5.BNC_XE8138Y1_1
DataFrame: TG5-LL-BNC5.BNC_XE8139X_1
DataFrame: TG5-LL-BNC5.BNC_XE8139Y_1


In [11]:
# Encontrar el rango de fechas común
min_date = dataframes["TG5-LL-BNA5.BNA_XE8132A_PT_4"]['Estampa de Tiempo'].min()
max_date = dataframes["TG5-LL-BNA5.BNA_XE8132A_PT_4"]['Estampa de Tiempo'].max()
# Crear un índice de tiempo común con intervalos de 1 segundo
time_index = pd.date_range(start=min_date, end=max_date, freq='S')

In [12]:
# Hacer datetime la estampa de tiempo
for name, df in dataframes.items():
    df['Estampa de Tiempo']=pd.to_datetime(df['Estampa de Tiempo'])

In [13]:
# Supongamos que tu diccionario de dataframes se llama `dataframes_dict`

for key, df in dataframes.items():
    # Asignar la columna 'Estampa de Tiempo' como índice
    df.set_index('Estampa de Tiempo', inplace=True)

# Ahora cada dataframe en el diccionario tiene la columna 'Estampa de Tiempo' como índice

In [14]:
import pandas as pd

# Supongamos que `dataframes_dict` es tu diccionario de dataframes 
# y `time_index` es un pd.DatetimeIndex con el nuevo índice de tiempo.

for key, df in dataframes.items():
    # Eliminar duplicados en el índice 'Estampa de Tiempo'
    df = df[~df.index.duplicated(keep='first')]
    
    # Reindexar el dataframe usando el nuevo 'time_index'
    df_reindexed = df.reindex(time_index)
    
    # Interpolar valores en la columna 'Variable'
    df_reindexed['Variable'] = df_reindexed['Variable'].interpolate(method='time')
    
    # Volver a llenar la columna 'TG05' con el valor constante original
    df_reindexed['*TG05*'] = df['*TG05*'].iloc[0]
    
    # Asignar el dataframe modificado de nuevo al diccionario
    dataframes[key] = df_reindexed

In [15]:
import pandas as pd

# Supongamos que dataframes_dict es tu diccionario de dataframes

# Paso 1: Unificar todos los dataframes en uno solo
df_unificado = pd.concat(dataframes.values())

# Paso 2: Crear el diccionario de equivalencias para cambiar los nombres en la columna TG05
equivalencias = {
    'LL-BNA5.BNA_XE8132A_PT_4': 'TG05 Estructura posterior Central BNA 8132 PT',
    'LL-BNA5.BNA_XE8134A_2': 'TG05 Cojinete de alineación J3 8134A',
    'LL-BNA5.BNA_XE8134B_2': 'TG05 Cojinete de alineación J3 8134B',
    'LL-BNA5.BNA_XE8134C_2': 'TG05 Cojinete de alineación J3 8134C',
    'LL-BNA5.BNA_XE8135A_2': 'TG05 Cojinete de alineación J4 8135A',
    'LL-BNA5.BNA_XE8135B_2': 'TG05 Cojinete de alineación J4 8135B',
    'LL-BNA5.BNA_XE8135C_2': 'TG05 Cojinete de alineación J4 8135C',
    'LL-BNB5.BNB_XE8131B_PT_4': 'TG05 Estructura Frontal Central  BNB 8131 PT',
    'LL-BNB5.BNB_XE8133B_PT_4': 'TG05 IPT BNB 8133 PT',
    'LL-BNC5.BNC_XE8138X1_1': 'TG05 Cojinete de alineación J3 Prox X',
    'LL-BNC5.BNC_XE8138Y1_1': 'TG05 Cojinete de alineación J3 Prox Y',
    'LL-BNC5.BNC_XE8139X_1': 'TG05 Cojinete de alineación J4 Prox X'
}

# Invertir el diccionario de equivalencias para el reemplazo
#equivalencias_invertidas = {v: k for k, v in equivalencias.items()}

# Paso 3: Reemplazar los valores en la columna TG05 utilizando las equivalencias
df_unificado['*TG05*'] = df_unificado['*TG05*'].replace(equivalencias)

# Mostrar el dataframe unificado con los nuevos valores en la columna TG05
df_unificado.head()

,*TG05*,Variable
2024-01-31 03:00:01,TG05 Estructura posterior Central BNA 8132 PT,0.148773
2024-01-31 03:00:02,TG05 Estructura posterior Central BNA 8132 PT,0.202179
2024-01-31 03:00:03,TG05 Estructura posterior Central BNA 8132 PT,0.240326
2024-01-31 03:00:04,TG05 Estructura posterior Central BNA 8132 PT,0.217438
2024-01-31 03:00:05,TG05 Estructura posterior Central BNA 8132 PT,NaN


In [16]:
display(df_unificado.index)

DatetimeIndex(['2024-01-31 03:00:01', '2024-01-31 03:00:02',
               '2024-01-31 03:00:03', '2024-01-31 03:00:04',
               '2024-01-31 03:00:05', '2024-01-31 03:00:06',
               '2024-01-31 03:00:07', '2024-01-31 03:00:08',
               '2024-01-31 03:00:09', '2024-01-31 03:00:10',
               ...
               '2024-08-07 19:36:15', '2024-08-07 19:36:16',
               '2024-08-07 19:36:17', '2024-08-07 19:36:18',
               '2024-08-07 19:36:19', '2024-08-07 19:36:20',
               '2024-08-07 19:36:21', '2024-08-07 19:36:22',
               '2024-08-07 19:36:23', '2024-08-07 19:36:24'],
              dtype='datetime64[ns]', length=213061992, freq=None)

In [17]:
# Crear una lista de máscaras y DataFrames filtrados con una columna adicional
df_filtrados = []
for i, (inicio, fin) in enumerate(zip(df_arranques['INICIO (-0,5 hs)'], df_arranques['FIN (+2 hs)'])):
    mask = (df_unificado.index >= inicio) & (df_unificado.index <= fin)
    df_filtrado = df_unificado[mask].copy()  # Crear una copia para evitar modificaciones in situ
    df_filtrado['Número de Arranque'] = i + 1  # Agregar la columna con el número de arranque
    df_filtrados.append(df_filtrado)

# Concatenar los DataFrames filtrados sin ignorar el índice original
df_final = pd.concat(df_filtrados)  # El índice datetime original se mantendrá


In [ ]:
# Set the index as a new column named "TimeStamp"
df_final['TimeStamp'] = df_final.index
df_final

In [18]:
df_final.to_csv('dataframe_tg05_filtrado_arranques_FINAL.csv', index=True)

In [25]:
df_final.columns

Index(['Unnamed: 0', '*TG05*', 'Variable', 'Número de Arranque'], dtype='object')

LEO DATAFRAME FINAL

In [21]:
df_final = pd.read_csv('dataframe_tg05_filtrado_arranques_FINAL.csv')

In [26]:
df_final.rename(columns={'Unnamed: 0': 'Timestamp'}, inplace=True)

In [27]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11116235 entries, 0 to 11116234
Data columns (total 4 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   *TG05*              object 
 2   Variable            float64
 3   Número de Arranque  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 339.2+ MB


In [28]:
# Lista de nombres de variables a filtrar
nombres_variables = [
'TG05 Estructura Frontal Central  BNA 8131 PT',
'TG05 Estructura posterior Central BNA 8132 PT',
'TG05 BNB 8133 PT',
'TG05 IPT BNB 8133 PT',
'TG05 Cojinete de alineación J3 8134A',
'TG05 Cojinete de alineación J3 8134B',
'TG05 Cojinete de alineación J3 8134C',    
'TG05 Cojinete de alineación J3 Prox X',
'TG05 Cojinete de alineación J3 Prox Y',
'TG05 Cojinete de alineación J4 8135A',
'TG05 Cojinete de alineación J4 8135B',
'TG05 Cojinete de alineación J4 8135C',
'TG05 Cojinete de alineación J4 Prox X',
'TG05 Cojinete de alineación J4 Prox Y',
]
 
# Filtrar el dataframe
df_final= df_final[df_final['*TG05*'].isin(nombres_variables)]
 
# Mostrar las primeras filas del dataframe filtrado para verificar
print(df_final.head())

             Timestamp                                         *TG05*  \
0  2024-02-04 14:26:37  TG05 Estructura posterior Central BNA 8132 PT   
1  2024-02-04 14:26:38  TG05 Estructura posterior Central BNA 8132 PT   
2  2024-02-04 14:26:39  TG05 Estructura posterior Central BNA 8132 PT   
3  2024-02-04 14:26:40  TG05 Estructura posterior Central BNA 8132 PT   
4  2024-02-04 14:26:41  TG05 Estructura posterior Central BNA 8132 PT   

   Variable  Número de Arranque  
0       NaN                   1  
1       NaN                   1  
2       NaN                   1  
3       NaN                   1  
4       NaN                   1  


FILTRO ARRANQUES CON FALLA DE INSTRUMENTO

In [29]:
df_final = df_final[df_final['Número de Arranque'] != 42]
df_final = df_final[~((df_final['Número de Arranque'] >= 24) & 
                       (df_final['Número de Arranque'] <= 32) & 
                       (df_final['*TG05*'] == 'TG05 Cojinete de alineación J3 Prox X'))]
df_final = df_final[~((df_final['Número de Arranque'] >= 33) & 
                       (df_final['Número de Arranque'] <= 43) & 
                       (df_final['*TG05*'] == 'TG05 Cojinete de alineación J3 Prox Y'))]



In [35]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9136015 entries, 0 to 11107233
Data columns (total 4 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   *TG05*              object 
 2   Variable            float64
 3   Número de Arranque  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 348.5+ MB


In [31]:
import plotly.express as px

# Suponiendo que ya tienes el dataframe `df_final`

# Lista de nombres únicos de variables en la columna TG05
variables_vibracion = df_final['*TG05*'].unique()
figuras = []

# Iterar sobre cada variable de vibración y crear un gráfico
for variable in variables_vibracion:
    # Filtrar el dataframe para la variable actual
    df_variable = df_final[df_final['*TG05*'] == variable]
    
    # Crear el gráfico de línea
    fig = px.line(
        df_variable,
        x=df_variable["Timestamp"],
        y='Variable',
        color='Número de Arranque',
        title=f'Comportamiento de {variable} en cada arranque',
        labels={'Numero_Arranque': 'Número de Arranque', 'Fecha': 'Tiempo', 'Variable': 'Vibración'},
        markers=True
    )

    # Personalizar colores: si el Número de Arranque es 1, usar rojo
    for trace in fig.data:
        if trace.name == '1':  # Comparar con la cadena '1'
            trace.line.color = 'red'
        else:
            trace.line.color = None  # Dejar el color predeterminado para los demás

    # Actualizar layout
    fig.update_layout(legend_title_text='Número de Arranque')
    
    # Añadir la figura a la lista
    figuras.append(fig)

# Exportar todas las figuras a un solo archivo HTML
with open("figuras_vibraciones.html", "a") as f:
    for fig in figuras:
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write('<br><hr><br>')  # Añadir una separación entre gráficos

print("Archivo HTML guardado como 'figuras_vibraciones.html'")

Archivo HTML guardado como 'figuras_vibraciones.html'


In [34]:
import plotly.express as px
import plotly.io as pio

# Lista de nombres únicos de variables en la columna TG05
variables_vibracion = df_final['*TG05*'].unique()

# Lista para almacenar las figuras
figuras = []

# Iterar sobre cada variable de vibración y crear un gráfico individual
for variable in variables_vibracion:
    # Filtrar el dataframe para la variable actual
    df_variable = df_final[df_final['*TG05*'] == variable]

    # Calcular la duración (tiempo transcurrido en segundos) para cada registro dentro de cada arranque
    df_variable = df_variable.sort_values(by=['Número de Arranque']).sort_index()
    #df_variable['Duración'] = df_variable.groupby('Número de Arranque').apply(lambda x: (x.index - x.index.min()).total_seconds())
    df_variable['Duración'] = df_variable.groupby('Número de Arranque')['Timestamp'].transform(lambda x: (x - x.min()).dt.total_seconds())
    # Crear el gráfico de línea con la fecha en el tooltip
    fig = px.line(
        df_variable,
        x='Duración',
        y='Variable',
        color='Número de Arranque',
        markers=True,
        labels={'Número de Arranque': 'Número de Arranque', 'Duración': 'Tiempo (segundos)', 'Variable': 'Vibración'},
        hover_data={df_variable.index.name: '|%Y-%m-%d %H:%M:%S'}  # Formato de la fecha en el pop-up
    )

    # Personalizar la apariencia de las líneas
    for trace in fig['data']:
        if trace.name == '1':  # Si el número de arranque es 1
            trace.line.color = 'red'  # Línea roja
            trace.marker.symbol = 'triangle-up'  # Marcador triángulo hacia arriba
        else:  # Para otros números de arranque
            trace.line.dash = 'dash'  # Línea punteada

    # Ajustar el layout y los títulos
    fig.update_layout(title=f'Comportamiento de {variable} en cada arranque', showlegend=True)

    # Configurar el eje x para mostrar segundos
    fig.update_xaxes(title_text="Tiempo de Duración (segundos)",
                     tickformat=',.0f')  # Formato con cero decimales

    # Añadir la figura a la lista
    figuras.append(fig)

# Exportar todas las figuras a un solo archivo HTML
with open("figuras_vibraciones_duracion.html", "a") as f:
    for fig in figuras:
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write('<br><hr><br>')  # Añadir una separación entre gráficos

print("Archivo HTML guardado como 'figuras_vibraciones_duracion.html'")


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

# Lista de nombres únicos de variables en la columna TG05
variables_vibracion = df_final['*TG05*'].unique()

# Crear una figura de subgráficos que contenga todos los gráficos
fig = make_subplots(rows=len(variables_vibracion), cols=1, shared_xaxes=True,
                    subplot_titles=[f'Comportamiento de {variable} en cada arranque' for variable in variables_vibracion])

# Iterar sobre cada variable de vibración y crear un gráfico
for i, variable in enumerate(variables_vibracion):
    # Filtrar el dataframe para la variable actual
    df_variable = df_final[df_final['*TG05*'] == variable]

    # Calcular la duración (tiempo transcurrido en segundos) para cada registro dentro de cada arranque
    df_variable = df_variable.sort_values(by=['Número de Arranque']).sort_index()
    df_variable['Duración'] = df_variable.groupby('Número de Arranque').apply(lambda x: (x.index - x.index.min()).total_seconds())

    # Crear el gráfico de línea con la fecha en el tooltip
    fig_aux = px.line(
        df_variable,
        x='Duración',
        y='Variable',
        color='Número de Arranque',
        markers=True,
        labels={'Número de Arranque': 'Número de Arranque', 'Duración': 'Tiempo (segundos)', 'Variable': 'Vibración'},
        hover_data={df_variable.index.name: '|%Y-%m-%d %H:%M:%S'}  # Formato de la fecha en el pop-up
    )

    # Personalizar la apariencia de las líneas
    for trace in fig_aux['data']:
        if trace.name == '1':  # Si el número de arranque es 1
            trace.line.color = 'red'  # Línea roja
            trace.marker.symbol = 'triangle-up'  # Marcador triángulo hacia arriba
        else:  # Para otros números de arranque
            trace.line.dash = 'dash'  # Línea punteada

    # Añadir el gráfico al subplot
    for trace in fig_aux['data']:
        fig.add_trace(trace, row=i+1, col=1)

# Ajustar el layout y los títulos
fig.update_layout(height=350*len(variables_vibracion), showlegend=True)

# Configurar el eje x para mostrar segundos
fig.update_xaxes(title_text="Tiempo de Duración (segundos)",
                tickformat=',.0f')  # Formato con cero decimales

# Guardar la figura completa en un archivo HTML
pio.write_html(fig, file='arranques_duracion_2.html')


In [ ]:
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

# Lista de nombres únicos de variables en la columna TG05
variables_vibracion = df_final['*TG05*'].unique()

# Crear una figura de subgráficos que contenga todos los gráficos
fig = make_subplots(
    rows=len(variables_vibracion), cols=1, shared_xaxes=True,
    subplot_titles=[f'Comportamiento de {variable} en cada arranque' for variable in variables_vibracion]
)

# Iterar sobre cada variable de vibración y crear un gráfico
for i, variable in enumerate(variables_vibracion):
    # Filtrar el dataframe para la variable actual
    df_variable = df_final[df_final['*TG05*'] == variable]
    
    # Crear el gráfico de líneas sin leyenda
    fig_aux = px.line(
        df_variable,
        x=df_variable.index,
        y='Variable',
        color='Número de Arranque',
        markers=True,
        labels={'Numero_Arranque': 'Número de Arranque', 'Fecha': 'Tiempo', 'Variable': 'Vibración'}
    )
    
    # Añadir el gráfico al subplot, ocultando la leyenda individual
    for trace in fig_aux['data']:
        trace.update(showlegend=(i == 0))  # Mostrar la leyenda solo en el primer subplot
        fig.add_trace(trace, row=i+1, col=1)

# Ajustar el layout y los títulos
fig.update_layout(
    height=300*len(variables_vibracion), 
    showlegend=True,
    legend_title_text='Número de Arranque'
)
fig.update_xaxes(title_text="Tiempo", row=len(variables_vibracion), col=1)

# Guardar la figura completa en un archivo HTML
pio.write_html(fig, file='todos_los_arranques_2.html')

# Mostrar la figura
fig.show()


In [ ]:
# Grafico en una misma linea temporal todos los arranques
import pandas as pd
import plotly.express as px
 
# Supongamos que tienes tu dataset en un DataFrame llamado 'df_final'
# df_final = pd.read_csv('tu_archivo.csv')
 
# Convertimos la columna "Estampa de Tiempo" a datetime
df_final['Estampa de Tiempo'] = pd.to_datetime(df_final['Estampa de Tiempo'])
 
# Normalizamos las estampas de tiempo a un rango de 0 a 9000
min_time_global = df_final['Estampa de Tiempo'].min()
max_time_global = df_final['Estampa de Tiempo'].max()
 
# Mapeamos todas las estampas de tiempo al rango 0-9000
df_final['ESTAMPA_NORMALIZADA'] = (
    (df_final['Estampa de Tiempo'] - min_time_global).dt.total_seconds() /
    (max_time_global - min_time_global).total_seconds() * 9000
)
 
# Ahora generamos un gráfico de líneas para cada valor único de *TG05*
for tg_value in df_final['*TG05*'].unique():
    df_tg = df_final[df_final['*TG05*'] == tg_value]
    fig = px.line(df_tg, x='ESTAMPA_NORMALIZADA', y='Variable',
                  color='Número de Arranque',
                  title=f'Gráfico para *TG05* = {tg_value}',
                  labels={'ESTAMPA_NORMALIZADA': 'Tiempo Normalizado (0-9000)'})
    fig.show()

 
 # Exportar todas las figuras a un solo archivo HTML
#with open("vibraciones_temporal.html", "a") as f:
#    for fig in figuras:
#        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#        f.write('<br><hr><br>')  # Añadir una separación entre gráficos

#print("Archivo HTML guardado como 'vibraciones_temporal.html'")
